# Regression Week 2: Multiple Linear Regression Assignment 1

## Predicting House Prices (Multiple Variables)

In this notebook you will use data on house sales in King County to predict prices using multiple regression. The first assignment will be about exploring multiple regression in particular exploring the impact of adding features to a regression and measuring error. In the second assignment you will implement a gradient descent algorithm. In this assignment you will:

- Use SFrames to do some feature engineering
- Use built-in GraphLab Create (or otherwise) functions to compute the regression weights (coefficients)
- Given the regression weights, predictors and outcome write a function to compute the Residual Sum of Squares
- Look at coefficients and interpret their meanings
- Evaluate multiple models via RSS

#### Import needed modules

In [1]:
import pandas as pd
import numpy as np
import sklearn.linear_model as LinearRegression
import zipfile
import os
from math import log
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

#### Unzip and load datasets (being careful of types)

In [2]:
with zipfile.ZipFile('kc_house_data.csv.zip', "r") as z:
    z.extractall(os.getcwd())
with zipfile.ZipFile('kc_house_test_data.csv.zip', "r") as z:
    z.extractall(os.getcwd())
with zipfile.ZipFile('kc_house_train_data.csv.zip', "r") as z:
    z.extractall(os.getcwd())
    
# Dictionary with the correct dtypes for the DataFrame columns
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 
              'sqft_living15':float, 'grade':int, 'yr_renovated':int, 
              'price':float, 'bedrooms':float, 'zipcode':str, 
              'long':float, 'sqft_lot15':float, 'sqft_living':float, 
              'floors':str, 'condition':int, 'lat':float, 'date':str, 
              'sqft_basement':int, 'yr_built':int, 'id':str, 'sqft_lot':int, 'view':int}
    
sales = pd.read_csv('kc_house_data.csv', dtype = dtype_dict)
train_data = pd.read_csv('kc_house_train_data.csv', dtype = dtype_dict)
test_data = pd.read_csv('kc_house_test_data.csv', dtype = dtype_dict)

#### Preview of train data

In [3]:
train_data.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900,3,1.00,1180,5650,1,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000,3,2.25,2570,7242,2,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000,2,1.00,770,10000,1,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000,4,3.00,1960,5000,1,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000,3,2.00,1680,8080,1,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


#### Define dependant features

In [4]:
train_data['bedrooms_squared'] = train_data['bedrooms']*train_data['bedrooms']
train_data['bed_bath_rooms'] = train_data['bedrooms']*train_data['bathrooms']
train_data['log_sqft_living'] = np.log(train_data['sqft_living'])
train_data['lat_plus_long'] = train_data['lat']+train_data['long']

In [5]:
print("Means\nbedrooms_squared: {0}\nbed_bath_rooms: {1}\nbedrooms_squared: {2}\nlat_plus_long: {3}\n".format(
    train_data['bedrooms_squared'].mean(),
    train_data['bed_bath_rooms'].mean(),
    train_data['log_sqft_living'].mean(),
    train_data['bedrooms_squared'].mean()))

Means
bedrooms_squared: 12.1742406811
bed_bath_rooms: 7.49659169351
bedrooms_squared: 7.55034931769
lat_plus_long: 12.1742406811



### Training a multiple regression model

**..**